# 1. Data Sourcing

To effectively address our research question — how undergraduate students at LSE might strategically select courses and degrees that offer relatively easier pathways to achieving high grades — we first need to acquire a range of publicly available data from LSE's online resources. This initial phase involves identifying relevant data sources, scraping and collecting data, exploring and cleaning said datasets from faulty and irrelevant information, as well as preparing and structuring the datasets for downstream processing and analysis.

The data sourcing process was divided into collecting two core datasets: (1) degree programme data, including recommended modules and application statistics, and (2) grade distributions by module, obtained from 5-year departmental PDF reports. These datasets need to be collected through targeted web scraping and manual extraction methods, considering the limitations of unstructured or semi-structured web formats.


### Data of interest
We specifically focused on the following data elements:
1. **Undergraduate Degrees 2024/25:**
    * Degree Names (of all available programmes)
    * Module Information
        * Mandatory Modules (per year)
        * Outside Module Options
    * Extra Information
        * A-level requirements
        * Application stats (volume, intake & acceptance rate)
        * Tuition fees
        * Median salary post-graduation
<br>
<br>
2. **Undergraduate Modules 2024/25:**
    * Module Codes (of all available courses)
    * Grade distributions
        * Grade summary statistics (mean, median, standard deviation, min, max, quartiles)
        * Classified grade distributions (Number of 1st, 2:1s, 2:2s, 3rds, fails)
    * Module Selection Criteria
        * Prerequisites for courses
        * Mutually exclusive courses
    * Extra Information
        * Number of enrolled students
        * Average class sizes
        * Capsizes (if applicable)
        * Units of courses
        * Responsible Departments

### Data sources
This data can be found on the following LSE websites:
1. **Degree Information and Application Statistics**
<br> *URL: https://www.lse.ac.uk/Programmes/Search-courses*
<br> (22 pages of degree programmes with individual programme pages containing module, entry, and application data)

2. **Module Grade Distributions**
<br> *URL: https://info.lse.ac.uk/staff/divisions/academic-registrars-division/systems/what-we-do/internal/degree-and-course-results*
<br> (includes departmental PDF files with annual module-grade data from 2019 - 2024)

3. **Course Guide and Module Metadata:** 
<br> *URL: https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides/undergraduate.htm*
<br> (Contains course unit values, prerequisites, exclusions, departments, and descriptions)


## 1.1. Degree Data Scraping

The first phase of data acquisition involved extracting relevant degree programme information from the LSE Degree Search platform. This site lists all available undergraduate programmes for the 2024/25 academic year across 22 paginated results. Each programme contains a detail page with structured data including:

* Programme title and UCAS code
* Recommended modules (year-wise, compulsory and optional)
* A-level entry requirements
* Application statistics (number of applicants, number of offers, and intake)
* Tuition fees (Home and International)
* Career outcomes (median salary post-graduation, if listed)
---

### 1.1.1. Degree webpage URL scraping

We first need to scrape the hyperlinks to all undergraduate degrees listed on the LSE Degree Search platform, which we will store in the programme_links list for further data extraction in later steps. Sorting through all 22 paginated result pages of the degree catalogue is achieved through extending the *Search-course* website's URL with it's page index and looping the link extraction through every results page. Within this loop the code uses requests to fetch each page’s HTML content and employs BeautifulSoup to parse the HTML and extract all anchor tags. Only links that correspond to undergraduate programme pages — identified by its URL pattern — are selected and converted into full URLs. These links are stored in the programme_links list for further data extraction in later steps.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

base_url = 'https://www.lse.ac.uk'
courses_url = 'https://www.lse.ac.uk/Programmes/Search-courses?pageIndex='

# Getting links to websites of all undergraduate programme
programme_links = []

for page in range(1, 23):
    print(f'Scraping page {page}/22...', end='\r', flush=True)
    url = f"{courses_url}{page}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    all_links = soup.find_all('a', href=True)
    
    for link in all_links:
        href = link['href']
        if href.startswith('/study-at-lse/undergraduate/'):
            full_url = base_url + href
            programme_links.append(full_url)

print('\n'+ f'Total undergraduate programmes found: {len(programme_links)}')

Scraping page 22/22...
Total undergraduate programmes found: 42


### 1.1.2. Degree data scraping - Function

With the list of programme URLs collected in the previous step, the next step is to extract structured information from each individual degree page. To achieve this, we define a function called scrape_programme_data, which takes a single URL as input and returns a dictionary of the degree’s attributes.

Before writing the function, it was necessary to manually inspect several programme webpages and their underlying HTML structures using developer tools and by examining the parsed output from BeautifulSoup. This allowed us to locate where elements of interest are stored within the HTML tree and identify consistent CSS selectors or tags we could use to extract the relevant data.

**Targeted elements:**
* **Degree name and department:** Retrieved from the page’s `<h1><span>` heading.
* **Course structure per academic year:** Modules are listed in div blocks identified by unique `#year-x` IDs. These were iterated over to capture the course codes year-by-year.
* **Entry requirements:** A-level requirements are stored under a specific element with the ID `#alevels`, and typically follow a paragraph structure.
* **Application statistics:** These appear as stylized bullet-point figures under the `"Your Application"` section and include number of applications, intake, and acceptance ratio.
* **Cost information:** Undergraduate home fees are usually listed in a paragraph block under the fees section and were extracted using a regular expression to capture pound-amounts `(£)`. Due to inconsistencies in the storage of overseas fees for various degrees, we opted to exclude this information from our dataset.
* **Median graduate salary:** If available, this is shown in the `"Graduate Destinations"` section.

The result of each function call is a dictionary containing all scraped data points, which can be appended to a list or converted into a DataFrame for further cleaning and analysis.

*Note, since LSE webpages occasionally returned temporary server errors (status code 500) during scraping, we added a status code check at the start of the function to skip any pages that could not be successfully accessed. This ensures our scraping process remains robust and continues running without interruption.*

*Further important observation is, that while the module lists are meant to only include all mandatory set courses for each year, in reality at times they include optional courses that are still listed on a degree's website as official course recommendations. Due to major inconsistencies among the storage of such recommended course options on the various websites, we are forced to later manually update these lists to exclude such instances and truly only reflect set mandatory courses for each degree.*

In [2]:
# Function for extraxting data of interest from websites
def scrape_programme_data(url):
    res = requests.get(url)
    if res.status_code != 200: # Needed to include due to temporary type 500 errors occuring when loading websites
        print(f"⚠️ Skipping {url} — status code {res.status_code}")
        return None
    
    soup = BeautifulSoup(res.text, 'html.parser')
    data = {}

    # Degree
    course = soup.select_one('h1 > span').get_text(strip=True)
    data['degree'] = course

    # A-level requirement
    alevel_elem = soup.select_one('#alevels > div > p')
    alevel_text = alevel_elem.get_text(strip=True).split(maxsplit=1)
    data['a_lvl_req'] = alevel_text[0].strip(',')
    if len(alevel_text) > 1: data['a_lvl_extra'] = alevel_text[1]
    else: data['a_lvl_extra'] = None

    # Modules (looping through years)
    data['modules_y1'] = []
    data['modules_y2'] = []
    data['modules_y3'] = []
    data['modules_y4'] = []
    for year in range(1, 5):  # assuming up to Year 3
        modules = soup.select(f'#year-{year} div.code')
        for module in modules:
            code = module.get_text(strip=True)
            if year == 1:
                data['modules_y1'].append(code)
            elif year == 2:
                data['modules_y2'].append(code)
            elif year == 3:
                data['modules_y3'].append(code)
            elif year == 4:
                data['modules_y4'].append(code)
        
    # Applications statistics
    nr_apps = soup.select_one("#your-application__overview .block--applications .stats")
    if nr_apps: data['nr_applications'] = nr_apps.get_text(strip=True)
    else: data['nr_applications'] = None
        
    intake = soup.select_one("#your-application__overview .block--places .stats")
    if intake: data['intake'] = intake.get_text(strip=True)
    else: data['intake'] = None
        
    ratio = soup.select_one("#your-application__overview .block--ratio .stats")
    if ratio: data['ratio'] = ratio.get_text(strip=True)
    else: data['ratio'] = None

    # Fees
    home_fee_text = soup.select_one('#fees-and-funding__home p').get_text(strip=True)
    data['home_fee'] = re.search(r'£[\d,]+', home_fee_text).group()

    # Median Salary
    salary = soup.select_one('#graduate-destinations__overview .salary')
    if salary: data['median_salary'] = salary.get_text(strip=True)
    else: data['median_salary'] = None

    return data

### 1.1.3. Degree data scraping - Application

We can now loop through each undergraduate degree URL and apply our previously defined scraping function to extract relevant data. Successfully scraped data is stored in a list, while any pages that failed to load (due to temporary server errors) are skipped and counted. This gives us a complete, structured dataset from the available programme pages.

In [3]:
# Applying function on all websites
degrees_data = []
skipped_urls = 0

for i, url in enumerate(programme_links):
    print(f"Scraping {i+1}/{len(programme_links)}: {url}", end='\r', flush=True)
    info = scrape_programme_data(url)
    if info is None:
        skipped_urls += 1
    else:
        degrees_data.append(info)

print('\nData scraping complete\n')
if skipped_urls >= 1:
    print(f'{skipped_urls} programmes skipped in data extraction, due to website loading error (500).')

⚠️ Skipping https://www.lse.ac.uk/study-at-lse/undergraduate/bsc-economic-history — status code 500conomics
Scraping 42/42: https://www.lse.ac.uk/study-at-lse/undergraduate/llb-bachelor-of-lawsogyehavioural-scienced-with-politicsss
Data scraping complete

1 programmes skipped in data extraction, due to website loading error (500).


### 1.1.4. Cleaning & Structuring Data

After scraping, we remove any `None` entries from the list—these correspond to non-responsive or failed websites. We then convert the cleaned list of dictionaries into a pandas DataFrame. This tabular format allows for easier inspection, manipulation, and analysis of the degree data moving forward.

In [4]:
# Cleaning data from non-responsive websites & converting to Dataframe
degrees_data_clean = [d for d in degrees_data if d is not None]
degrees_df = pd.DataFrame(degrees_data_clean)
degrees_df.head(3)

,degree,a_lvl_req,a_lvl_extra,modules_y1,modules_y2,modules_y3,modules_y4,nr_applications,intake,ratio,home_fee,median_salary
0,BA Anthropology and Law,AAB,None,"[LL141, AN100, AN101, LL142, LL108, LL100, LL1...","[AN253, AN379, LL106, LL143, LL200]",[LL276],[],250,20,13:1,"£9,535","£34,500"
1,BA Geography,AAA,None,"[GY100, GY140, GY144, LSE100]","[GY245, GY246, GY212, GY204, GY206, GY207]",[GY350],[],377,38,10:1,"£9,535","£35,000"
2,BA History,AAA,None,"[HY120, LSE100, EH101, HY113, HY116, HY118]",[],[HY300],[],503,58,9:1,"£9,535","£35,000"


### 1.1.5. Exporting the Data

To preserve our cleaned dataset and enable easy reuse in future analysis steps, we save the DataFrame as a CSV file in our project directory. This allows us to avoid re-scraping the web every time we need the data.

In [5]:
# Saving as CSV file
degrees_df.to_csv('data/degrees/programme_data.csv', index=False)
print('Data has been saved as a CSV file')

Data has been saved as a CSV file


This CSV file now forms a foundational part of our analysis, offering essential context for identifying potential patterns in course difficulty, grade distributions, and academic outcomes across different degree programmes.

## 1.2. Module Grade Distribution Scraping

To complement our degree-level data and gain more insight into what makes a course “easy” or high-scoring, we now turn to the course-level grade distributions. These statistics are published annually by each department at LSE in the form of PDF documents, which include a breakdown of student performance in each undergraduate module — typically showing distribution statistics and frequencies such as mean and median grades, as well as classified results (e.g., 1sts, 2:1s, etc.) for the last 5 years running.

Unfortunately, the LSE webpage where these PDFs are hosted is behind a login portal that requires student credentials. Since it is not publicly accessible and protected against automated scraping, we manually downloaded the full set of course results PDFs across all departments and stored them locally in the *data/modules* folder. This ensures we can still extract and analyze the grade data programmatically.

The goal of this section is to loop through each of these PDFs, parse out the relevant statistics for each undergraduate module, and clean them into a structured format suitable for analysis.

---

### 1.2.1. Identifying Departments PDF's
We start by looping through all files and identifying all relevant PDF files in our folder, allowing us to extract the module-level data needed for our broader analysis.

In [6]:
import pdfplumber
import os
import contextlib
import io

# Identifying all PDFs
pdf_folder = 'data/modules'
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]
total_files = len(pdf_files)

### 1.2.2. Extracting text from PDF

Unlike scraping data from HTML websites — where structured elements like tags, classes, and IDs help us pinpoint exactly where data is stored — the structure of PDFs is often less consistent and not inherently designed for data extraction. As such, we needed to use the specialized Python library `pdfplumber`. This allows us to read and parse text content from PDF files while preserving the layout and line structure of the original documents.

The following function reads every page of a given PDF and concatenates all extracted text into a single string, preparing it for further pattern-based filtering and analysis.

*Note, at times the parsing encountered insignificant layout or font issues, resulting in persistent warning messages displayed - eventhough the output was perfectly fine. To deal with this and keep our output clean and readable, we imported the contextlib and io libraries. These let us suppress standard error messages during the PDF processing step. This workaround ensures that we can extract the text content reliably while ignoring any non-critical warnings cluttering the output.*

In [7]:
# Function to extract text from PDF using pdfplumber
def extract_pdf_text(pdf_path):
    all_text = ''
    with contextlib.redirect_stderr(io.StringIO()): # Use of AI to avoid warning messages
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    all_text += page_text + '\n'
    return all_text

### 1.2.3. Identifying Departments

To begin parsing the content of each module PDF, we first needed to identify which department the module data belongs to. To do this, we created a function that scans the extracted PDF text line by line, searching for a pattern that consistently appears across files:
"*Department (XY) course results*".

Due to rare inconsistencies in the pattern of a course code (e.g. courses like ST101A or EC2C3 deviating from the typical AB123 pattern), we used the `re` library and AI to help write a generic regular expression that looks for a department name followed by a short code in parentheses and hence can generically match this format flexibly across departments and various course codes.

We also used the `.group()`-function to extract the matching parts of the string (i.e. the department name and code). This function was something we discovered and learned to apply through AI assistance.

The final result is a tuple containing the department code and department name, which we’ll use to label and organize the extracted data correctly.

In [8]:
# Function to extract department name & code
def extract_department_code(text):
    lines = text.split('\n')
    
    for line in lines:
        match = re.search(r'([A-Za-z]+) \(([A-Za-z]{2,4})\) course results', line) # Use of AI to generate generic re code that identifies string
        if match:
            department_name = match.group(1) # Use of AI to learn about .group() function
            department_code = match.group(2)
            break
    return department_code, department_name

### 1.2.4. Collecting Grade Distribution Statistics

Next, we focus on parsing the actual Marksummary tables from each PDF. These tables contain statistical data such as mean, median, standard deviation of marks, as well as minimum and maximum marks and percentile values about student performance across individual courses and academic years. This data allows us to assess performance patterns across departments and over time and are the key part of our module-level analysis.

**Approach to Extraction**

We extract the data by defining a function that uses a while loop to parse through each PDF line-by-line using an index variable i, looking for the start of tables marked by the consistent header string *'Year marks mean sd'*. Once the header is found, we:
* store the column names and map them to their respective indices
* check for a course code, often found a few lines below the table, formatted like *AB123:Marksummary*
* loop through each row, parsing values and adding them to a structured list mark_data (if the row is complete and aligns with the header)
* keep track of excluded rows in excluded_data, especially those that are either empty or misaligned.

This setup allows us to extract data even when PDFs contain multiple tables or have slightly inconsistent formatting.


**Challenges Encountered and Fixes**

One major issue we ran into was dealing with incomplete or misaligned table rows. These rows typically arise when the PDF text parser encounters blank cells in the original table—commonly seen when there are 0 values (e.g., 0 students received a fail grade). Unfortunately, when using pdfplumber (or any other PDF parser), these blank cells are not interpreted as 0, but instead are skipped altogether, causing the rest of the row to shift left, misaligning values with the headers and hence having the total number of entries in a row to fall short of the header length.

This issue was especially common in the second table (called *'Gradesummary'*) that follows the Marksummary for each course, which contains the degree classification frequencies (e.g., # & % of students who got 1st, 2:1, 2:2, fail). Despite extensive attempts using both extensive online research and AI suggestions, we were not able to reliably parse this table due to the unpredictability of missing values and their impact on row structure.

Thus, we made the decision to exclude the second table from our dataset. While unfortunate — since classification frequencies offer valuable and arguably more interesting insight — this decision was necessary to maintain the integrity of our dataset. The Marksummary statistics, by contrast, are consistently populated (as statistical summaries like mean, median, and standard deviation always require numerical input) and hence were extracted successfully in most cases. Including only rows where the lenght of values matches the lenght of the header, fully ensures that our dataset is reliable and clean.

In [9]:
# Function to extract Marksummary tables from text
def extract_marksummary(text, department_code, department_name):
    lines = text.split('\n')
    mark_data = []
    excluded_data = []

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Identifying table headers
        if line.startswith('Year marks mean sd'):
            header = line.split()
            pos = {col: idx for idx, col in enumerate(header)}

            # Identifying course code (at bottom of table)
            course = department_code # Setting department code as default
            for k in range(1, 7):
                if i + k < len(lines):
                    match = re.search(r'([A-Z0-9]+):Marksummary', lines[i + k])
                    if match:
                        course = match.group(1)
                        break

            # Moving to first data row (skipping header)
            i += 1
            course_data = []
            skipped_rows = []
            
            while i < len(lines):
                line = lines[i].strip()
                # Break when encountering table title (at bottom of each table)
                if re.match(r'([A-Z0-9]+):Marksummary', line) or line.startswith('MarksbyYear'):
                    break

                # Parsing data
                if line:
                    values = line.split()
                    if len(values) == len(pos): # cleaning data from incomplete and misaligned rows due to missing values
                        course_data.append({
                            'department': department_name,
                            'code': course,
                            'year': values[pos['Year']],
                            'marks': int(values[pos['marks']]),
                            'mean': float(values[pos['mean']]),
                            'sd': float(values[pos['sd']]),
                            'min': float(values[pos['min']]),
                            'q10': float(values[pos['q10']]),
                            'q25': float(values[pos['q25']]),
                            'median': float(values[pos['median']]),
                            'IQR': float(values[pos['IQR']]),
                            'q75': float(values[pos['q75']]),
                            'q90': float(values[pos['q90']]),
                            'q95': float(values[pos['q95']]),
                            'max': float(values[pos['max']])
                        })
                        
                    elif len(values) == 1: # Seperating excluded rows between empty rows and incomplete rows
                        skipped_rows.append({'course': course, 'year': values[pos['Year']], 'reason': 'no data in year'})
                        
                    else:
                        skipped_rows.append({'course': course, 'year': values[pos['Year']], 'reason': 'incomplete data'})
                
                            
                i += 1  # Moving to next line

            mark_data.extend(course_data)
            excluded_data.extend(skipped_rows)

        else:
            i += 1  # Moving to next line

    return mark_data, excluded_data

### 1.2.5. PDF Processing & Application

Finally, we need to bring together all previously defined steps and apply them to every PDF in the specified folder with the following function. For each file, the mark summary statistics are parsed from the text, and valid rows are appended to a main data list, while excluded or malformed rows are collected separately. Lastly, both datasets are returned as DataFrames, ready for subsequent processing and analysis.

In [10]:
# Function to scrape all PDFs in the folder
def process_pdfs(pdf_folder):
    all_data = []
    all_excl_data = []
    for i, pdf_file in enumerate(pdf_files, 1):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        print(f'Processing ({i}/{total_files}): {pdf_file}...', end='\r', flush=True)

        # Extracting text from the PDF
        text = extract_pdf_text(pdf_path)
        
        # Extracting department code and name
        department_code, department_name = extract_department_code(text)

        # Extracting mark summary data
        mark_data, excluded_data = extract_marksummary(text, department_code, department_name)
        
        # Appending data
        all_data.extend(mark_data)
        all_excl_data.extend(excluded_data)

    # Converting data
    df = pd.DataFrame(all_data)
    df_excl = pd.DataFrame(all_excl_data)
    
    return df, df_excl

### 1.2.6. Sorting Dataframe & Identifying Excluded Rows

Once all PDFs have been processed, the resulting data is sorted by course code and academic year for easier readability and analysis. The DataFrame index is reset to ensure consistency after sorting. In addition, we report how many rows were excluded due to either being completely empty (often corresponding to years before a course was introduced) or misaligned (typically caused by missing values within a table row).

In [11]:
# Scraping all PDFs & sorting data
df, df_excl = process_pdfs(pdf_folder)
df = df.sort_values(by=['code', 'year'], ascending=[True, True])
df.reset_index(drop=True, inplace=True)

empty_rows = len(df_excl[df_excl['reason'] == 'no data in year'])
misaligned_rows = len(df_excl[df_excl['reason'] == 'incomplete data'])

print('\n'+f'Data scraping complete, {len(df)} rows of data extracted.'+'\n')
print(f'{empty_rows} rows deleted due to empty rows for years prior to introduction of new modules.')
print(f'{misaligned_rows} rows deleted due to missing values resulting in misalignment.')

Processing (20/20): DS-results-2023-24-All-Sittings.pdf...
Data scraping complete, 1934 rows of data extracted.

516 rows deleted due to empty rows for years prior to introduction of new modules.
49 rows deleted due to missing values resulting in misalignment.


### 1.2.7. Saving Seperated Dataframes

In the final step of the module data scraping process, we distinguish between department-level and individual module-level data based on the length of the course code,creating two separate DataFrames for clearer organization and analysis. Each is then saved as a CSV file in the appropriate directory, ensuring our data is both accessible and structured for the next phase of the project.

In [12]:
# Separating modules and department data
departments_df = df[df['code'].str.len() == 2]
departments_df.reset_index(drop=True, inplace=True)

modules_df = df[df['code'].str.len() > 2]
modules_df.reset_index(drop=True, inplace=True)

# Saving DataFrames to CSV files
modules_df.to_csv("data/modules/marks_summary_modules.csv", index=False)
departments_df.to_csv("data/departments/marks_summary_departments.csv", index=False)

print('Dataframes seperated and saved as CSV files')
modules_df.head(3)

Dataframes seperated and saved as CSV files


,department,code,year,marks,mean,sd,min,q10,q25,median,IQR,q75,q90,q95,max
0,Accounting,AC100,2019/20,116,76.9,9.2,45.0,65.0,72.8,79.0,10.5,83.2,86.0,87.0,91.0
1,Accounting,AC100,2020/21,145,67.4,10.4,32.0,54.0,61.0,69.0,14.0,75.0,79.0,81.0,85.0
2,Accounting,AC100,2021/22,114,65.8,15.6,0.0,47.3,58.0,68.0,19.0,77.0,81.7,85.7,88.0


## 1.3. Module Extra Info Scraping

To complement the statistical data we extracted from the departmental mark summaries, we also collect additional contextual information about each module from the official LSE course guides.

Unlike the PDFs, the course guide data is published on a public-facing HTML website, which means we are able to scrape this data automatically using Python. Hence we can scrape the master course guide index page to retrieve links to all available undergraduate modules for the academic year 2024/25 and afterwards visit each course’s individual page to extract key pieces of structured information.

**Elements of extra information of interest:**
* Full Course Name
* Responsible Department
* Prerequisites for taking the course
* Total number of students enrolled (in the previous year)
* Average class size
* Course Cap Size (if applicable)
* Academic unit (i.e. how many credits it's worth)

These data points allow us to better contextualize our quantitative mark statistics by providing metadata on how each module is structured.

---

### 1.3.1. Collecting Course Guides

To start, we scrape the main undergraduate course guide page to collect URLs for all individual course modules. After sending a request to the page and parsing it with BeautifulSoup, we extract all anchor tags within tables — where we found the course links grouped by department through HTML inspection. Then we can filter for valid guide links (those starting with *"../courseGuides/"*) and reconstruct each full URL using the known base path. The resulting `course_links` list contains the direct URLs to all undergraduate course pages.

In [116]:
import numpy as np

base_url = 'https://www.lse.ac.uk/resources/calendar2024-2025/courseGuides'
guide_url = f'{base_url}/undergraduate.htm'

# Getting all links to course guides
response = requests.get(guide_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Finding all tables (each course is stored in departments table)
tables = soup.find_all('table')

course_links = []

for table in tables:
    for a_tag in table.find_all('a', href=True):
        href = a_tag['href']
        if href.startswith('../courseGuides/'):
            full_url = base_url + href.split('../courseGuides')[1]
            course_links.append(full_url)

print(f'Found {len(course_links)} course guide links.')

Found 568 course guide links.


### 1.3.2. Identifying Extra Info

Next, we define a function that takes a single course guide URL and scrapes all data of interest from it. After extracting the course code and title from the `<title>` tag, it then accesses the `“Key Facts”` section and pulls out relevant fields like department name, student numbers, class size, whether the course is capped, and unit value. Each of these is identified by the prefix of the `<p>` tag’s content (e.g. “Department:”, “Value:”, etc.).

The second part of the function focuses on identifying prerequisite modules. It searches the `"Prerequisites"` section and uses a regular expression to extract any valid LSE course codes. Note, that once again due to rare inconsistencies in the pattern of a course code (e.g. courses like ST101A or EC2C3 deviating from the typical AB123 pattern), we used AI to create a generic regular expression that includes such irregularities. To further avoid circular references, we filter out duplicates and the course itself. All extracted data is stored in a dictionary, ready to be appended to a larger dataset.

In [117]:
def extract_course_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    title = soup.find('title').get_text().split(maxsplit=1)
    code = title[0]
    course = title[1]
    
    data = {'code': code, 'course': course}
    
    # Extracting Information from Key Facts Section
    key_facts_section = soup.find('div', id='keyFacts-Content')
    items = key_facts_section.find_all('p')
        
    for item in items:
        text = item.get_text(strip=True)
    
        if text.startswith('Department'):
            data['department'] = text.split(':')[1].strip()
        elif text.startswith('Total students'):
            data['total_students'] = text.split(':')[1].strip()
        elif text.startswith('Average class size'):
            data['avg_class_size'] = text.split(':')[1].strip()
        elif text.startswith('Capped'):
            data['capped'] = text.split(':')[1].strip()
        elif text.startswith("Value:"):
            data['units'] = text.split('Value:')[1].strip()
    
    # Extracting Information from Prerequisits Section
    prereq_div = soup.find('div', id='preRequisites-Content')
    prereqs = set()
    
    if prereq_div:
        text = prereq_div.get_text(separator=' ', strip=True)
        prereqs.update(re.findall(r'\b([A-Z]{2,}\d+[A-Z]?)\b', text)) # Use of AI to create generic re string
        prereqs.discard(code)

    data['prerequisites'] = list(prereqs)
        
    return data

### 1.3.3. Scraping Extra Info

Now that we have set up the scraping function, we can run the scraper across all previously collected course guide links, printing progress along the way. The extracted data is then converted into a dataframe, as can be seen below.

In [128]:
# Running the scraper
all_course_data = []

for i, url in enumerate(course_links):
    print(f"Scraping {i+1}/{len(course_links)}: {url}", end='\r', flush=True)
    course_data = extract_course_data(url)
    all_course_data.append(course_data)

# Converting to DataFrame
modules_facts = pd.DataFrame(all_course_data)
modules_facts[modules_facts['prerequisites'].astype(bool)].head(3)

,code,course,department,total_students,avg_class_size,capped,units,prerequisites
4,AC205,Intermediate Financial Accounting,Accounting,Unavailable,Unavailable,No,Half Unit,"[AC102, AC105]"
5,AC206,Intermediate Management Accounting,Accounting,Unavailable,Unavailable,No,Half Unit,"[AC106, AC103]"
6,AC311,Results Accountability and Management Control ...,Accounting,118,39,No,Half Unit,"[AC103, AC200, AC312, AC100]"


### 1.3.4. Cleaning and Formatting

Next we clean and standardize the scraped course data to prepare it for analysis. Unit values are converted from strings to numerical floats. Missing values in student counts and average class sizes are handled by replacing "Unavailable" with NaN and casting to floats. The capped field is converted to a boolean or numeric value depending on whether a cap exists, extracting the numerical cap where present. Finally, the 'code' column is set as the index for the DataFrame to uniquely identify each module.

In [129]:
# Cleaning Data
modules_facts['units'] = modules_facts['units'].map({'One Unit': 1.0, 'Half Unit': 0.5, 'Non-credit bearing': 0.0})
modules_facts['total_students'] = modules_facts['total_students'].replace('Unavailable', np.nan).astype(float)
modules_facts['avg_class_size'] = modules_facts['avg_class_size'].replace('Unavailable', np.nan).astype(float)
modules_facts.loc[modules_facts['capped'].str.startswith('No'), 'capped'] = False
modules_facts.loc[modules_facts['capped'] != False, 'capped'] = modules_facts.loc[modules_facts['capped'] != False, 'capped'].str.split(' ').str[1].str.strip('()').astype(int)
modules_facts.set_index('code', inplace=True)
modules_facts.head(3)

,course,department,total_students,avg_class_size,capped,units,prerequisites
code,,,,,,,
AC102,Elements of Financial Accounting,Accounting,564.0,15.0,False,0.5,[]
AC103,"Elements of Management Accounting, Financial M...",Accounting,256.0,18.0,False,0.5,[]
AC105,Introduction to Financial Accounting,Accounting,115.0,39.0,False,0.5,[]


### 1.3.5. Mutually Exclusive Modules

We are further interested in courses that are mutually exclusive to other courses, as we need to confirm valid course selection later on in our analysis. Thus, we manually created a CSV file containing all mutually exclusive course pairs, which we load into a dataframe, setting the 'code' column as the index for easier lookups. Printing out the dataframe info, confirms that there are 62 mutually exclusive pairs.

*Note, that manual creation of this file was necessary due to major complications in the way information on mutual exclusive courses was stored in the course guides, with pairs often only mentioned in textblocks and their exclusivity explained with 'and' or 'or' statements, of which reliable data extraction would have superseeded our coding abilities.*

In [130]:
mutually_exclusive = pd.read_csv('data/modules/mutual_exclusive.csv')
mutually_exclusive.set_index('code', inplace=True)
mutually_exclusive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, DS101A to ST330
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mutually_exclusive  62 non-null     object
dtypes: object(1)
memory usage: 992.0+ bytes


### 1.3.6. Merging Dataframes

With our mutually exclusive data, we can now merge the two DataFrames by performing a left join based on the `code` column. This ensures that the mutually exclusive course data is added as a new column in the `modules_facts` DataFrame, with matching values for the corresponding courses.

In [131]:
# Add mutually exclusive course data as a new column
modules_facts = modules_facts.join(mutually_exclusive, how='left')

# Display updated DataFrame
modules_facts[modules_facts['mutually_exclusive'].notna()].head()

,course,department,total_students,avg_class_size,capped,units,prerequisites,mutually_exclusive
code,,,,,,,,
DS101A,Fundamentals of Data Science,Data Science Institute,28.0,11.0,False,0.5,[],DS101W
DS101W,Fundamentals of Data Science,Data Science Institute,8.0,5.0,False,0.5,[],DS101A
DS105A,Data for Data Science,Data Science Institute,60.0,13.0,False,0.5,[],DS105W
DS105W,Data for Data Science,Data Science Institute,52.0,10.0,False,0.5,[],DS104A
EC1A3,Microeconomics I,Economics,751.0,16.0,False,0.5,[],EC1A5


### 1.3.7. Validating Cleaned Data

To ensure data integrity before storage, we inspect the structure and completeness of the `modules_facts` DataFrame using `.info()`. The output shows that all modules have complete values for most columns, with missing values in `total_students` and `avg_class_size` due to unreported statistics in the course guides—these omissions are not critical for our analysis.

All entries contain valid values in the prerequisites column, confirming that our scraper consistently extracted prerequisite information, even where no prerequisites were listed (resulting in empty lists).

We also identify that only 56 out of the 62 mutually exclusive course pairs were successfully merged. By comparing the indices and manually inspecting the course guides, we confirm that the 6 missing course codes are absent from the course guides altogether, suggesting they correspond to outdated or discontinued modules. Since these are no longer offered, they are irrelevant for our current analysis and can be safely ignored.

In [138]:
# Counting rows containing NaN values
print(modules_facts.info())

# Identifying missing mutual exclusive pairs
missing_codes = set(mutually_exclusive.index) - set(modules_facts.index)
print('\n'+"Missing codes:", missing_codes)
print(f"{len(missing_codes)} out of {len(mutually_exclusive)} total entries are missing.")

# Saving to CSV
modules_facts.to_csv('data/modules/modules_key_facts.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 569 entries, AC102 to ST360
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   course              569 non-null    object 
 1   department          569 non-null    object 
 2   total_students      468 non-null    float64
 3   avg_class_size      461 non-null    float64
 4   capped              569 non-null    object 
 5   units               569 non-null    float64
 6   prerequisites       569 non-null    object 
 7   mutually_exclusive  56 non-null     object 
dtypes: float64(3), object(5)
memory usage: 56.2+ KB
None

Missing codes: {'SO208', 'SO308', 'SO210', 'FM213', 'DS202', 'SO224'}
6 out of 62 total entries are missing.


## 1.4. Outside Options

Finally, we need to compile a list of standard outside options — elective modules available to most undergraduate students across various LSE programmes. While there is no centralized list explicitly outlining all shared outside options, manual inspection across different programme regulations (especially BSc Finance) reveals a consistent set of course codes frequently permitted as electives. We use the BSc Finance programme page to extract this list as a reliable proxy, which will be crucial for later analysis of elective pathways and degree flexibility.

---

### 1.4.1. Identifying Outside Options List 

We begin by inspecting BSc Finance undergraduate programme page and parsing its HTML content with BeautifulSoup, identifying that this page contains a table listing the standard outside options allowed for most degrees.

In [154]:
url = 'https://www.lse.ac.uk/resources/calendar/programmeRegulations/undergraduate/2024/BScFinance.htm'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

### 1.4.2. Locating & Extracting Course Codes

We iterate through all `<h3>` tags to locate the one linking to the Undergraduate Outside Options List, then identify the surrounding `<div>` and extract all course codes from the associated table. Upon inspection of the resulting dataframe we can count 374 elective modules that can be chosen outside a student’s main department.

In [155]:
for h3 in soup.find_all("h3"):
    a_tag = h3.find("a")
    if a_tag and "Undergraduate Outside Options List" in a_tag.text:
        header = h3
        break

outside_course_codes = []

parent_div = header.find_parent("div")
table = parent_div.find("table")
links = table.find_all("a", href=True)
for row in table.find_all("tr"):
    link = row.find("a", href=True)
    if link:
        course_code = link.text.strip()
        outside_course_codes.append(course_code)
        
outside_options = pd.DataFrame(outside_course_codes)
outside_options.columns = ['code']
outside_options.set_index('code', inplace=True)
outside_options.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, AC102 to ST330
Empty DataFrame


### 1.4.3. Merging Dataframes

We enrich the `outside_options` table by joining it with the `modules_facts` DataFrame to save a final dataset that solely contains all extra information on outside options.

In [156]:
outside_options = outside_options.join(modules_facts, how='left')
outside_options.to_csv('data/degrees/ug_outside_options.csv', index=False)
outside_options.head()

,course,department,total_students,avg_class_size,capped,units,prerequisites,mutually_exclusive
code,,,,,,,,
AC102,Elements of Financial Accounting,Accounting,564.0,15.0,False,0.5,[],NaN
AC103,"Elements of Management Accounting, Financial M...",Accounting,256.0,18.0,False,0.5,[],NaN
AC205,Intermediate Financial Accounting,Accounting,NaN,NaN,False,0.5,"[AC102, AC105]",NaN
AC206,Intermediate Management Accounting,Accounting,NaN,NaN,False,0.5,"[AC106, AC103]",NaN
AC311,Results Accountability and Management Control ...,Accounting,118.0,39.0,False,0.5,"[AC103, AC200, AC312, AC100]",NaN


## 1.5. Creating Complete Dataset